In [1]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [386]:
data = pd.read_csv("botswana.tsv", sep="\t", engine = 'python') 

In [387]:
data

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,0,16,7,protestant,2.0,1.0,0.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4357,2,28,7,protestant,4.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4358,4,24,5,protestant,4.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4359,1,26,0,spirit,5.0,1.0,0.0,1,22.0,7.0,0,0.0,1.0,0.0,0.0


In [388]:
data.drop(['evermarr'], inplace = True, axis = 1)

nevermarr = data['agefm'].map({float('NaN') : 1})
nevermarr = nevermarr.map({float('NaN') : 0, 1 : 1})
data.insert(value = nevermarr, column = 'nevermarr', loc = 7)
data['agefm'].fillna(0, inplace = True)
data.loc[data['nevermarr'].isin([1]),'heduc'] = -1

In [389]:
data['heduc'].unique()

array([-1., 14.,  1., 12.,  7., 11.,  9., 17.,  3., 16., nan, 10., 15.,
        0., 13.,  8.,  4.,  6., 20.,  5.,  2., 19., 18.])

In [390]:
data['agefm'].fillna(0, inplace = True)

In [391]:
idlnchld_noans = data['idlnchld'].map({float('NaN') : 1})
idlnchld_noans = idlnchld_noans.map({float('NaN') : 0, 1 : 1})

heduc_noans = data['heduc'].map({float('NaN') : 1})
heduc_noans = heduc_noans.map({float('NaN') : 0, 1 : 1})

usemeth_noans = data['usemeth'].map({float('NaN') : 1})
usemeth_noans = usemeth_noans.map({float('NaN') : 0, 1 : 1})

In [392]:
data.insert(value = idlnchld_noans, column = 'idlnchld_noans', loc = 4)
data.insert(value = heduc_noans, column = 'heduc_noans', loc = 10)
data.insert(value = usemeth_noans, column = 'usemeth_noans', loc = 7)

data.loc[data['idlnchld_noans'].isin([1]),'idlnchld'] = -1
data.loc[data['heduc_noans'].isin([1]),'heduc'] = -2
data.loc[data['usemeth_noans'].isin([1]),'usemeth'] = -1

In [395]:
data['idlnchld'].unique()

array([ 4.,  2.,  3.,  5.,  6.,  1., -1.,  0., 11., 10.,  8.,  7., 20.,
        9., 14., 17., 12., 15., 16., 13.])

In [396]:
data.dropna(inplace = True)

In [405]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld_noans + idlnchld + knowmeth + usemeth_noans + usemeth + nevermarr + agefm + heduc_noans + heduc + urban + electric + radio + tv + bicycle', 
             data=data)
fitted1 = m1.fit()
print(fitted1.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Fri, 24 Apr 2020   Prob (F-statistic):               0.00
Time:                        16:31:40   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [406]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted1.resid, fitted1.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [407]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld_noans + idlnchld + knowmeth + usemeth_noans + usemeth + nevermarr + agefm + heduc_noans + heduc + urban + electric + radio + tv + bicycle', 
             data=data)
fitted1 = m1.fit(cov_type='HC1')
print(fitted1.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Fri, 24 Apr 2020   Prob (F-statistic):               0.00
Time:                        16:31:49   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [408]:
m2 = smf.ols('ceb ~ age + educ + idlnchld_noans + idlnchld + knowmeth + usemeth_noans + usemeth + nevermarr + agefm + heduc_noans + heduc + urban + electric + bicycle', 
             data=data)
fitted2 = m2.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Fri, 24 Apr 2020   Prob (F-statistic):               0.00
Time:                        16:31:59   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [409]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted2.resid, fitted2.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [410]:
m2 = smf.ols('ceb ~ age + educ + idlnchld_noans + idlnchld + knowmeth + usemeth_noans + usemeth + nevermarr + agefm + heduc_noans + heduc + urban + electric + bicycle', 
              data=data)
fitted = m2.fit(cov_type='HC1')
print(fitted.summary())


                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Fri, 24 Apr 2020   Prob (F-statistic):               0.00
Time:                        16:32:05   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [411]:
print("F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit()))

F=0.919236, p=0.467231, k1=5.000000


In [413]:
m3 = smf.ols('ceb ~ age + educ + idlnchld_noans + idlnchld + knowmeth + nevermarr + agefm + heduc_noans + heduc + urban + electric + bicycle', 
              data=data)
fitted3 = m3.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Fri, 24 Apr 2020   Prob (F-statistic):               0.00
Time:                        16:32:28   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [414]:
print("F=%f, p=%f, k1=%f" % m2.fit().compare_f_test(m3.fit()))

F=92.890582, p=0.000000, k1=2.000000


In [415]:
m2.fit().compare_f_test(m3.fit())

(92.89058230109757, 3.1552009480386492e-40, 2.0)